##### Copyright 2020 The TensorFlow Quantum Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Graph Recurrent Neural Networks for Ising model

Author : Jae H. Yoo, Google Research.

Contributors : Guillaume Verdon (X company) Trevor McCourt, Antonio J. Martinez, Michael Broughton (Google Research)

Created : 2020-Feb-01

Last updated : 2020-Feb-02 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tensorflow/quantum/blob/research/qgrnn_ising/qgrnn_ising.ipynb)

## Introduction

In this colab, we will learn how to train QGRNN, a variant of Quantum Graph Neural Networks ([Verdon et al.](https://arxiv.org/abs/1909.12264)) to learn the dynamics of the target Hamiltonian of given Ising model. Before going further, we will install related libraries.

### Import & pip install libraries

In [ ]:
!pip install --upgrade tensorflow==2.3.0

In [ ]:
!pip install tensorflow-quantum

In [ ]:
import networkx as nx
import numpy as np
import sympy
import cirq
# TensorFlow
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
# TensorFlow Quantum
import tensorflow_quantum as tfq
from tensorflow_quantum.python.layers import Expectation

import matplotlib.pyplot as plt
%matplotlib inline

### Overview

Here are steps we will follow.

- Preparation of Quantum Data with VQE on Ising model.
- Construct a QGRNN model
- Construct Fidelity with Swap Test
- Calculate the average infidelity loss function
- Train the QGRNN & get the final result


## Preparation of Quantum Data with VQE on Ising model

We will construct a target Hamiltonian $H_{target}$ by using a random ring graph $G$ with $N=6$ qubits. On top of it, we will find a low energy state near to ground state of $H_{target}$ by using Variational Quantum Eigensolver (VQE).

In [ ]:
# Define the number of qubits of our target quantum system.
N = 6

### Transverse field Ising model Hamiltonian

Here are basic introduction of the target Hamiltonian of Ising model. Given
* $J_{jk}$ for interacting spin pairs and
* $B_{v}$ for site bias term of each spin,

$H_{target} = \sum_{j,k} J_{jk} Z_j Z_k + \sum_{v} B_v Z_v + \sum_{v} X_v$

It is very easy to construct this Hamiltonian using networkx library in Python.

In [ ]:
G = nx.cycle_graph(N)
weights = [4*(np.random.random()-0.5) for _ in G.edges]
biases = [4*(np.random.random()-0.5) for _ in G.nodes]

Also, networkx provides graphic APIs to draw graph data. You can see that nodes have Brown to Blue Green colormap, and edges have Red to Blue one.

In [ ]:
def draw(graph, pos, weights, biases, title):
    label = {i:'{}'.format(i) for i in graph.nodes}
    edge_options = {
        "edge_color": weights,
        "width": 4,
        "edge_cmap": plt.cm.RdBu,
        "edge_vmin" : -2,
        "edge_vmax" : 2,
    }
    node_options = {
        "node_color": biases,
        "cmap": plt.cm.BrBG,
        "vmin" : -2,
        "vmax" : 2,
    }
    nx.draw_networkx_labels(graph, pos, label, font_color="w")
    nodes = nx.draw_networkx_nodes(graph, pos, **node_options)
    edges = nx.draw_networkx_edges(graph, pos, **edge_options)
    edges.set_cmap(plt.cm.RdBu)
    edges.set_clim(-2, 2)

    plt.title(title)
    plt.colorbar(nodes)
    plt.colorbar(edges)
    plt.show()


pos = nx.circular_layout(G)
draw(G, pos, weights, biases, 'Target Ising model')
print(*zip(G.edges, weights))
print(*zip(G.nodes, biases))

Now that we have the graph structure, weights of edges & nodes, we can construct `cirq` based Hamiltonian which can be directly calculated in `cirq` and `tfq`. To create Hamiltonian by using `cirq.PauliSum`'s or `cirq.PauliString`'s we need to assign appropriate qubits on them. We can bring qubits by using `cirq.GridQubit`.

In [ ]:
def Hamiltonian(graph, weights, biases, qubits):
    H_cost = [w * cirq.Z(qubits[i]) * cirq.Z(qubits[j]) \
            for (i, j), w in zip(graph.edges, weights)]
    H_cost += [b * cirq.Z(qubits[v]) for v, b in enumerate(biases)]
    H_mixer = [cirq.X(q) for q in qubits]
    return H_cost, H_mixer

qubits = cirq.GridQubit.rect(1, N)
true_H_cost, true_H_mixer = Hamiltonian(G, weights, biases, qubits) 
for cost in true_H_cost:
    print(cost)
for mixer in true_H_mixer:
    print(mixer)

### Variational Quantum Eigensolver (VQE)

Here, we will construct a Variational Quantum Eigensolver (VQE) to find out a low energy state $|\psi_0\rangle$ near to the ground state.

#### Variational method

[Variational method](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) provides the way to find approximated ground energy states. Let $|\psi(\vec\theta)\rangle$ be a variational ansatz. We can control parameters $\vec\theta$ to change the quantum state. Then, the following inequality is guaranteed for any given Hamiltonian $H$.

- $\langle\psi(\vec\theta)|H|\psi(\vec\theta)\rangle \ge E_0$, where $E_0$ is the energy of ground state.
- We can find out $\vec\theta$ giving us approximated ground state by minimizing the above expectation value as a loss function

#### Variational Ansatz

We can make any quantum state by using $X$ rotation & $Z$ rotation in Bloch sphere, which could be a simple and useful variational ansatz for our problem. Because our ansatz will be used for Ising model & QGRNN later, we should not assign qubits to the rotational gates because qubits are immutable in `cirq.Circuit`. That's why we use `VQE.gates` and `VQE.circuit` separately.


In [ ]:
class VQE:
    """Variational Quantum Eigensolver"""
    def __init__(self, qubits):
        self.qubits = qubits
        # Parameters
        t_x = ['t_x{}'.format(i) for i, _ in enumerate(qubits)]
        t_z = ['t_z{}'.format(i) for i, _ in enumerate(qubits)]
        self.symbols = t_x + t_z
        # Parameterized quantum gates without Qubits
        gates = []
        gates.append([cirq.XPowGate(exponent=sympy.Symbol(x)) for x in t_x])
        gates.append([cirq.ZPowGate(exponent=sympy.Symbol(z)) for z in t_z])
        self._gates = gates
        self.circuit = self.get_state(qubits)

    def get_state(self, qubits, params=None):
        """Outputs quantum data with given qubits."""
        circuit = cirq.Circuit(
            [g(i) for gates in self._gates for i, g in zip(qubits, gates)])
        if params is None:
            return circuit
        resolver = {k: v for k, v in zip(self.symbols, params)}
        return cirq.resolve_parameters(circuit, resolver)

vqe = VQE(qubits)
print(vqe.circuit)

#### Construct $\langle \psi (\vec\theta) | H | \psi (\vec\theta)\rangle$

TensorFlow Quantum provides `tfq.layers.Expectation` Keras layer to provide easy interface to calculate the expectation value of given ansatz & Hamiltonian.

In [ ]:
def vqe_keras_model(vqe, H_target):
    # Construct measurement layers.
    circuit_input = Input(shape=(), dtype=tf.string)
    output = Expectation()(
            circuit_input,
            symbol_names=vqe.symbols,
            operators=tfq.convert_to_tensor([H_target]),
            initializer=tf.keras.initializers.RandomNormal())
    # Each term in H_target is calculated respectively.
    # Here, we sum them up to get the final <H>.
    output = tf.math.reduce_sum(output, axis=-1, keepdims=True)

    # Model compile
    model = Model(inputs=circuit_input, outputs=output)
    adam = Adam(learning_rate=0.05)
    model.compile(optimizer=adam, loss='mse')
    return model

H_target = true_H_cost + true_H_mixer
model = vqe_keras_model(vqe, H_target)

#### Minimizing $\langle \psi (\vec\theta) | H | \psi (\vec\theta)\rangle$

Keras model is used for training VQE. To feed quantum data in the form of `cirq.Circuit`, `tfq.convert_to_tensor()` will be frequently used to make them `tf.Tensor`.

We have quantum input data. What's the output true value? Isn't it just a minimization problem?

Here is some tip for training. Setting the output true value to theoretical lower bound, we can minimize our expectation value in the Keras model fit framework. The how can we calculate the lower bound? By the fact that the expectation value of any PauliString is bounded in [-1, 1], we can easily find the lower bound.

- $\langle \psi (\vec\theta) | H | \psi (\vec\theta)\rangle = \sum_{jk}J_{jk}\langle Z_jZ_k\rangle + \sum_{v}B_{v}\langle Z_v\rangle + \sum_{v}\langle X_v\rangle \ge \sum_{jk}(-)|J_{jk}| -\sum_{v}|B_{v}| - N $

In [ ]:
lower_bound = -np.sum(np.abs(weights + biases)) - N
vqe_input = tfq.convert_to_tensor([vqe.circuit])
vqe_output = tf.convert_to_tensor([[lower_bound]])
print('Before training : <H>={}'.format(model.predict(x=vqe_input)))
history = model.fit(x=vqe_input, y=vqe_output, batch_size=1, epochs=100,
                    verbose=0)
plt.plot(history.history['loss'])
print('After training : <H>={} >= {}'.format(model.predict(x=vqe_input),
                                             lower_bound))

In the next sections, we will use this low energy state as initial states of Ising model & QGRNN. Since they have different qubits, we need to create both.

In [ ]:
vqe_params = model.get_weights()[0]
low_energy_ising = vqe.get_state(qubits, vqe_params)
# For QGRNN, get new qubit indices (0, N)~(0,2*N-1)
qubits_qgrnn = cirq.GridQubit.rect(1, N, 0, N)
low_energy_qgrnn = vqe.get_state(qubits_qgrnn, vqe_params)
print(low_energy_ising)
print(low_energy_qgrnn)

### Construct Ising model

In VQE, we can just use Hamiltonian $H$ directly. However, we need to exponentiate Hamiltonians to construct Ising model. Moreover, it is proved that exponentiation of two non-commutable observables is not equal to exponentiation of the sum of them. That is, $e^{A+B} \neq e^Ae^B$ if $[A, B]\neq0$.

There are two options to deal with this problem.
1. [Baker-Campbell-Hausdorff (BCH) formula](https://en.wikipedia.org/wiki/Baker%E2%80%93Campbell%E2%80%93Hausdorff_formula)

  - $e^Ae^B=e^Ae^Be^{\frac{1}{2}[A, B]}e^{\frac{1}{12}[A, [A, B]]}e^{-\frac{1}{12}[B, [A, B]]} ...$
  - Analytic solution, but it is hard to be calculated.

2. [Suzuki-Trotter expansion](https://en.wikipedia.org/wiki/Time-evolving_block_decimation#The_Suzuki-Trotter_expansion)

  - $e^Ae^B \simeq \prod e^{\delta t A}e^{\delta t B}$
  - Tractable, but it is an approximated solution

We will use the second option. Let $P=\frac{T_j}{\delta t}$.

  - $|\psi_{T_j}\rangle = U^{j}_{Ising}|\psi_0\rangle = e^{-i T_j H_{target}}|\psi_0\rangle\sim [\prod^{P}e^{-i {\delta t}H_{mixer}} e^{-i {\delta t}H_{cost}}]|\psi_0\rangle$

In [ ]:
class IsingModel:

    def __init__(self, qubits, graph, weights, biases, eta=0.01):
        self.qubits = qubits
        self.graph = graph
        self.weights = weights
        self.biases = biases
        self.eta = eta

        # Construct Hamiltonian
        _weights = [eta * w for w in weights]
        _biases = [eta * b for b in biases]
        H_cost, H_mixer = Hamiltonian(graph, _weights, _biases, qubits)
        # Exponentiate each Hamiltonian
        # The reason why we split cost & mixer is they are not commutable
        self._cost_step = tfq.util.exponential(operators=H_cost)
        self._mix_step = tfq.util.exponential(operators=H_mixer)

    def __call__(self, input_state, depth):
        """Trotterizaiton"""
        add = tfq.layers.AddCircuit()
        output_state = add(cirq.Circuit(), append=input_state)
        for _ in range(depth):
            output_state = add(output_state, append=self._cost_step)
            output_state = add(output_state, append=self._mix_step)
        return output_state

ising = IsingModel(qubits, G, weights, biases)

### Time evolution of Ising model
Let's construct an Ising model and evolve $|\psi_0\rangle$ with randomly sampled timesteps $T_j\in [0, T_{max}]$

In [ ]:
def random_timestep_sample(batch_size, T_max=0.1):
    return [T_max * np.random.uniform() for _ in range(batch_size)]

batch_size = 15
T = random_timestep_sample(batch_size)

We can evolve the above `low_energy_ising` and `low_energy_qgrnn` by using exponentiation of Hamiltonian & Trotterization. If we say $P$ is a Trotterization depth with time unit ${\delta t}$, we can get the depth $P=\frac{T}{\delta t}$

In [ ]:
dt = 0.01
depth = [int(t/dt)+1 for t in T] # Circuit depth for Ising & QGRNN model
print(depth)

true_final_states = []
for P in depth:
    true_final_states.append(ising(low_energy_ising, depth=P))

Now that we have a set $\{(|\psi_0\rangle, |\psi_{T_j}\rangle) | j = 1..M\}$ where $M$ is the number of data, or batch size, we finished to generate quantum data for QGRNN.

## Construct QGRNN model

In this section, we will implement QGRNN model with 

- Trainable parameters : $\theta_{jk}$, $\phi_{v}$, $\alpha_{v}$ 
  - $\alpha_{v}$ can be set to a constant 1.
- $H_{QGRNN} = \sum_{j,k} \theta_{jk} Z_j Z_k + \sum_{v} \phi_v Z_v + \sum_{v} \alpha_v X_v$

Because the target Hamiltonian is unknown to QGRNN, we need to initialize a new random graph inside our QGRNN. In this example we will use 4-regular graph (each node has 4 edges. For 6 nodes, graph has 12 edges.) But, for simplicity, we add missing edges in the ring structure.

In [ ]:
class QuantumGraphRNN:

    def __init__(self, qubits, G, eta=0.01, train_mixer=False):
        self.qubits = qubits
        self.eta = eta
        self.graph = G
        # Set parameters for cost
        self._theta = ['theta{}'.format(e) for e in G.edges]
        self._phi = ['phi{}'.format(v) for v in G.nodes]
        self.symbols = self._theta + self._phi 
        # Set parameters for mixer
        self.train_mixer = train_mixer
        if train_mixer:
            self._alpha = ['alpha{}'.format(v) for v in G.nodes]
            self.symbols += self._alpha
        else:
            self._alpha = [1.0 for _ in G.nodes]

        # Construct Hamiltonian
        weights = [eta] * len(G.edges)
        biases = [eta] * len(G.nodes)
        H_cost, H_mixer = Hamiltonian(G, weights, biases, qubits)
        # Exponentiate each Hamiltonian with parameters
        self._cost_step = tfq.util.exponential(
            operators=H_cost, coefficients=self._theta + self._phi)
        self._mix_step = tfq.util.exponential(
            operators=H_mixer, coefficients=self._alpha)

    def __call__(self, input_state, depth):
        add = tfq.layers.AddCircuit()
        output_state = add(cirq.Circuit(), append=input_state)
        for _ in range(depth):
            output_state = add(output_state, append=self._cost_step)
            output_state = add(output_state, append=self._mix_step)
        return output_state

# The true graph of Ising model is unknown to QGRNN.
# Think a new 4-regular random graph with at least one cycle
# 0->1->2->3->4->5->0
G_qgrnn = nx.random_regular_graph(n=N, d=4)
node_color = [2.0 for _ in G_qgrnn.nodes]
missing = []
add_cycle = True
if add_cycle:
    for i in range(N):
        j = (i+1) % N
        if (i, j) in G_qgrnn.edges or (j, i) in G_qgrnn.edges:
            continue
        print('add : ', i, j)
        G_qgrnn.add_edge(i, j)
        missing.extend([(i, j), (j, i)])
edge_color = [-2.0 if e in missing else 2.0 for e in G_qgrnn.edges]
draw(G_qgrnn, pos, edge_color, node_color, '4-regular random graph with cycle')
qgrnn = QuantumGraphRNN(qubits_qgrnn, G_qgrnn)

Let's evolve quantum input data $|\psi_0\rangle$ according to QGRNN ansatz, and get $U^{j}_{QGRNN}(\theta, \phi)|\psi_0\rangle$

In [ ]:
pred_final_states = []
for P in depth:
    pred_final_states.append(qgrnn(low_energy_qgrnn, depth=P))

## Construct Fidelity with Swap Test

Now we have both (1) quantum data from true Hamiltonian of Ising model and (2) predicted quantum data from QGRNN. Because they are all quantum data, we can't compare them each other as we do in the calculation of loss function at the classical deep neural networks. Measurement on a qubit will destory other qubit informations. That's why we introduce Fidelity with [Swap Test](https://en.wikipedia.org/wiki/Swap_test) here.

- The expectation value of swap test of two quantum states $|\psi\rangle$ and $|\phi\rangle$ is the square of the inner product of them.
  - $\operatorname{Prob}(0)=\operatorname{Prob}(Z=+1)=\frac{1}{2}+\frac{1}{2}|\langle\phi|\psi\rangle|^2$
  - $\langle Z_{test} \rangle = 1 \times \operatorname{Prob}(Z=+1) + (-1) \times \operatorname{Prob}(Z=-1)=2\operatorname{Prob}(Z=+1) - 1$
  - $\therefore \langle Z_{test} \rangle = |\langle\phi|\psi\rangle|^2$
- We have ground truth Ising model state $|\psi_{T_j}\rangle$ and predicted state $U^{j}_{QGRNN}(\theta, \phi)|\psi_0\rangle$
    - That is, $\langle Z_{test} \rangle_j = |\langle \psi_{T_j} | U^{j}_{QGRNN}(\theta, \phi)|\psi_0\rangle|^2 \ge 0$

As you can see, the expectation has lower bound 0.


In [ ]:
class SwapTestFidelity:

    def __init__(self, qubits1, qubits2, batch_size):
        circuit = cirq.Circuit()
        test_bit = set(cirq.GridQubit.rect(1, len(qubits1) + len(qubits2) + 1))
        test_bit -= set(qubits1 + qubits2)
        test_bit = list(test_bit)[0]
        circuit.append(cirq.H(test_bit))
        for a, b in zip(qubits1, qubits2):
            circuit.append(cirq.decompose(cirq.CSwapGate().on(test_bit, a, b)))
        circuit.append(cirq.H(test_bit))
        self.circuit = tfq.convert_to_tensor([circuit] * batch_size)
        self.op = tfq.convert_to_tensor([[cirq.Z(test_bit)]] * batch_size)

    def __call__(self, input_state1, input_state2):
        add = tfq.layers.AddCircuit()
        return add(add(input_state1, append=input_state2), append=self.circuit)
        
fidelity = SwapTestFidelity(qubits, qubits_qgrnn, batch_size)

# Construct measurement layers.
state_true = Input(shape=(), dtype=tf.string)
state_pred = Input(shape=(), dtype=tf.string)
fid_output = fidelity(state_true, state_pred)
fid_output = Expectation()(fid_output,
                           symbol_names=qgrnn.symbols,
                           operators=fidelity.op)

## Calculate the average infidelity

In fact, QGRNN is also a recurrent neural network that learns various time step evolution from the same input. We need to aggregate the results among the different timestep evolutions to train the QGRNN model. Here we introduce the average fidelity.

  - $L(\theta, \phi) = 1 - \frac{1}{B} \sum^{B}_{j=1} |\langle \psi_{T_j} | U^{j}_{QGRNN}(\theta, \phi)|\psi_0\rangle|^2 = 1 - \frac{1}{B} \sum^{B}_{j=1} \langle Z_{test} \rangle_j $

We can implement this custom keras loss function like this.

In [ ]:
def average_fidelity(y_true, y_pred):
    return 1 - K.mean(y_pred)

## Train the QGRNN & get the final result

Again, we can use Keras model fit. To feed a batch of quantum data, we can use `tf.concat` because the quantum circuits are already in `tf.Tensor`. In this case, we know that the lower bound of fidelity is 0, but the true `model_output` is not used in our custom loss function `average_fidelity`. For the purpose of comparison, `initial_params` contains the initial weights & biases of QGRNN. We set learning rate of Adam optimizer to $0.05$ as our paper described.

In [ ]:
# Model compile
model = Model(inputs=[state_true, state_pred], outputs=fid_output)
adam = Adam(learning_rate=0.05)
model.compile(optimizer=adam, loss=average_fidelity)
initial_params = model.get_weights()

# Data preparation
y_true = tf.concat(true_final_states, axis=0)
y_pred = tf.concat(pred_final_states, axis=0)
model_input = [y_true, y_pred]
# Lower bound of average fidelity = 0, but not used in average_fidelity
model_output = tf.convert_to_tensor([[0]] * batch_size)

In [ ]:
%%time
history = model.fit(x=model_input,
                    y=model_output,
                    batch_size=batch_size,
                    epochs=500)

In [ ]:
# Learning curve
plt.plot(history.history['loss'])
plt.title('Learning curve (min loss={:.3e})'.format(
    np.min(history.history['loss'])))
plt.xlabel('iteration')
plt.ylabel('log fidelity')
plt.yscale('log')

In [ ]:
# Target Ising model graph
draw(G, pos, weights, biases, 'Target Ising model')
# Initial QGRNN graph.
init_edge_color = initial_params[0][:len(G_qgrnn.edges)]
init_node_color = initial_params[0][len(G_qgrnn.edges):]
draw(G_qgrnn, pos, init_edge_color, init_node_color, 'Initial QGRNN model')
# Trained QGRNN graph.
final_params = model.get_weights()
final_edge_color = final_params[0][:len(G_qgrnn.edges)]
final_node_color = final_params[0][len(G_qgrnn.edges):]
draw(G_qgrnn, pos, final_edge_color, final_node_color, 'Trained QGRNN model')

# Conclusion

In this colab, we implemented a variant of Quantum Graph Neural Networks called Quantum Graph `Recurrent` Neural Network. As the name suggests, this model learns time evolution dynamics of given target Hamiltonian from its quantum data. You can modify and train your own configurations of the model. Here are some suggestions for you.

- Set `train_mixer=True` in class `QuantumGraphRNN` and retrain the model. Then the mixer Hamiltonian in QGRNN becomes also trainable. What can you observe after training?

- We added missing edges in the cycle `0->1->...->5->0`. What if we remove the cycle? It means the QGRNN doesn't know any clue of the true Hamiltonian at all except the number of qubits. Set `add_cycle=False`. Is the loss still going to the order of `1e-4`?

